# Using Virtual Environments

- Difference between virtual environments (virtualenv or venv), containers, and virtual machines
- Creating a virtual environment
- Activating and deactivating virtualenvs
- Installing packages in a virtual environment
- Using requirements.txt files for reproducability
- Using wheels for faster installations
- Cleaning up virtualenvs

# Virtual Machines, Containers, and Virtualenvs

## Virtual Machine

- Isolated image ("guest") of a computer running its own operating system
- Can have different OS than the host
- Examples: vmware, virtualbox, Amazon EC2

## Container

- Partially isolated environment that shares the operating system with the host
- Same OS kernel as host, but with potentially different users/libraries/networks/etc. and system limits
- Examples: Docker, Heroku, Amazon ECS

## Virtualenv

- Partially isolated **python** environment running in the same OS as the 'host'
- **Only** Python packages and environment are isolated:  basically a "private copy of Python"
- No _security_ isolation from host OS: a program running inside a virtualenv can do whatever a program running outside a virtualenv can do
- Similar to local `node_modules` subfolder in a project for Javascript developers

# Why Virtualenvs?

- **Dependency management** - If two different Python applications require two different versions of the same package, running each app in its own virtualenv allows both versions to be available to their respective applications
- **Keeps your system Python pristine** - Many OSes use Python to implement some of the OS tooling (RedHat in particular). This often results in an older version of Python, or particuar versions of Python packages installed globally that you *should not modify* if you want your system tools to keep working.
- **Helps with reproducibility** - Virtualenvs allow you to note the versions of all packages installed in your venv in order to recreate the virtualenv on another machine. This prevents the "Works on My Machine" certification.

# Creating a virtualenv


## Installing virtualenv

Since Python 3.3, Python has included a tool to create virtual environments called `venv` in the standard library. 

If, however, you are developing on Ubuntu, you must separately install it anyway with `apt-get install python-venv`.



## Creating the virtualenv

To create a virtual environment, you invoke the `venv` module with the virtualenv name:

```shell
$ python -m venv env-folder
```

This command

- creates a folder named `env-folder`
- copies the Python you used to invoke `venv` into that folder
- creates a couple of helper scripts inside env-folder to activate/deactivate the virtualenv

In [1]:
!which python

/Users/rick446/.virtualenvs/py37/bin/python


In [2]:
!python -m venv data/env-folder

We can see the directory structure that the virtual environment created with the `tree` command. If you don't have `tree`, you can install it on a Mac using homebrew:

```bash
$ brew install tree
```

In [3]:
!tree -d data/env-folder

data/env-folder
├── bin
├── include
└── lib
    └── python3.7
        └── site-packages
            ├── __pycache__
            ├── pip
            │   ├── __pycache__
            │   ├── _internal
            │   │   ├── __pycache__
            │   │   ├── cli
            │   │   │   └── __pycache__
            │   │   ├── commands
            │   │   │   └── __pycache__
            │   │   ├── distributions
            │   │   │   └── __pycache__
            │   │   ├── models
            │   │   │   └── __pycache__
            │   │   ├── operations
            │   │   │   └── __pycache__
            │   │   ├── req
            │   │   │   └── __pycache__
            │   │   ├── utils
            │   │   │   └── __pycache__
            │   │   └── vcs
            │   │       └── __pycache__
            │   └── _vendor
            │       ├── __pycache__
            │       ├── cachecontrol
            │       │   ├── __pycache__
            │       │   └── caches
            │      

In [4]:
ls data/env-folder/bin

activate          easy_install*     pip3*             python3@
activate.csh      easy_install-3.7* pip3.7*
activate.fish     pip*              python@


### (windows note)

If you are using Windows, there should be a `Scripts` folder under the environment folder instead of `bin`, and it should contain an `activate.bat` file.

You can invoke the Python in your new virtualenv by specifying the full path:

In [5]:
!data/env-folder/bin/python --version

Python 3.7.6


In [6]:
!data/env-folder/bin/python -c 'import sys; print(sys.executable)'

/Users/rick446/src/arborian-classes/data/env-folder/bin/python


## Activating virtual environments

More commonly, we will *activate* the virtualenv for our current shell by `source`-ing the `activate` script

### Linux

```shell
$ source env-folder/bin/activate
(env-folder) $
```

or

```shell
$ . env-folder/bin/activate
(env-folder) $
```

### Windows

```shell
c:\...> env-folder\Scripts\activate.bat
(env-folder) c:\...>
```

Activating the virtualenv does a few things to your *current shell/terminal window only*:

- Puts the virtualenv's executable folder (`bin` or `Scripts`) at the beginning of your path so the virtualenv python will be picked up automatically
- Changes your prompt so you see that you are in the virtualenv
- Makes a `deactivate` command available to undo the changes

## Deactivating virtual environments

### Linux

```shell
(env-folder) $ deactivate
$
```

### Windows

```shell
(env-folder) c:\...> deactivate
c:\...> 
```


In [7]:
%%bash
source data/env-folder/bin/activate
which python
echo My prompt is now $PS1
python -c 'import sys; print(sys.executable)'
deactivate
which python
python -c 'import sys; print(sys.executable)'

/Users/rick446/src/arborian-classes/src/data/env-folder/bin/python
My prompt is now (env-folder)
/Users/rick446/src/arborian-classes/data/env-folder/bin/python
/Users/rick446/.virtualenvs/py37/bin/python
/Users/rick446/.virtualenvs/py37/bin/python


# Installing packages in virtual environments

When the virtual environment is activated, or when you invoke the version of Python in the virtualenv, you can install third-party packages into the virtualenv without modifying your system Python:

In [8]:
%%bash
set -e
source data/env-folder/bin/activate
which python
pip install -U pip
python -m pip install numpy
python -c 'import numpy; print(numpy)'

/Users/rick446/src/arborian-classes/src/data/env-folder/bin/python
Looking in links: /Users/rick446/src/wheelhouse
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3
Looking in links: /Users/rick446/src/wheelhouse
<module 'numpy' from '/Users/rick446/src/arborian-classes/data/env-folder/lib/python3.7/site-packages/numpy/__init__.py'>


# Using requirements.txt for reproducibility

Once you have your app in your virtualenv running, you may need to reproduce the virtualenv on another machine. 
`pip` has a command `freeze` which outputs the exact versions of all packages installed in a virtualenv:

In [9]:
%%bash
set -e
source data/env-folder/bin/activate
python -m pip freeze

-f /Users/rick446/src/wheelhouse
numpy==1.18.2


Normally, we'll put this into a file `requirements.txt` that we check into source control and distribute with our project:

In [10]:
%%bash
set -e
source data/env-folder/bin/activate
pip freeze > data/requirements.txt

Once we have the requirements.txt file, we can create a new virtualenv and install all the same versions of packages into it:

In [11]:
%%bash
set -e
python -m venv data/env-folder-2
source data/env-folder-2/bin/activate
python -m pip install -r data/requirements.txt

Looking in links: /Users/rick446/src/wheelhouse, /Users/rick446/src/wheelhouse
  Using cached https://files.pythonhosted.org/packages/81/14/6d7c914dac1cb2b596d2adace4aa4574d20c0789780f1339d535e69e271f/numpy-1.18.2-cp37-cp37m-macosx_10_9_x86_64.whl


You should consider upgrading via the 'pip install --upgrade pip' command.


# Using wheels for faster installations

While `pip` tries to cache as much data as possible, we can do even better by using "wheels." 

Wheels are Python packages that have been compiled (if necessary) for a particular target architecture and are thus much faster to install. 

If you're moving to a new machine (for instance, when deploying to production) it can also be useful to have the wheels cached locally so `pip` doesn't try to download the packages from the Python Package Index.

In [12]:
%%bash
set -e
source data/env-folder/bin/activate
pip install scipy sklearn jupyter simplejson pymongo boto3 wheel
pip freeze > data/requirements.txt

Looking in links: /Users/rick446/src/wheelhouse
  Using cached scipy-1.4.1-cp37-cp37m-macosx_10_6_intel.whl (28.4 MB)
Processing /Users/rick446/src/wheelhouse/sklearn-0.0-py2.py3-none-any.whl
Processing /Users/rick446/src/wheelhouse/jupyter-1.0.0-py2.py3-none-any.whl
  Using cached simplejson-3.17.0-cp37-cp37m-macosx_10_14_x86_64.whl (73 kB)
  Using cached pymongo-3.10.1-cp37-cp37m-macosx_10_9_x86_64.whl (350 kB)
Processing /Users/rick446/src/wheelhouse/wheel-0.34.2-py2.py3-none-any.whl
  Using cached scikit_learn-0.22.2.post1-cp37-cp37m-macosx_10_9_x86_64.whl (7.1 MB)
Processing /Users/rick446/src/wheelhouse/notebook-6.0.3-py3-none-any.whl
Processing /Users/rick446/src/wheelhouse/nbconvert-5.6.1-py2.py3-none-any.whl
  Using cached qtconsole-4.7.1-py2.py3-none-any.whl (117 kB)
Processing /Users/rick446/src/wheelhouse/ipywidgets-7.5.1-py2.py3-none-any.whl
Processing /Users/rick446/src/wheelhouse/jupyter_console-6.1.0-py2.py3-none-any.whl
  Using cached jmespath-0.9.5-py2.py3-none-any.wh

In [13]:
cat data/requirements.txt

-f /Users/rick446/src/wheelhouse
appnope==0.1.0
attrs==19.3.0
backcall==0.1.0
bleach==3.1.3
boto3==1.12.26
botocore==1.15.26
decorator==4.4.2
defusedxml==0.6.0
docutils==0.15.2
entrypoints==0.3
importlib-metadata==1.5.0
ipykernel==5.2.0
ipython==7.13.0
ipython-genutils==0.2.0
ipywidgets==7.5.1
jedi==0.16.0
Jinja2==2.11.1
jmespath==0.9.5
joblib==0.14.1
jsonschema==3.2.0
jupyter==1.0.0
jupyter-client==6.1.0
jupyter-console==6.1.0
jupyter-core==4.6.3
MarkupSafe==1.1.1
mistune==0.8.4
nbconvert==5.6.1
nbformat==5.0.4
notebook==6.0.3
numpy==1.18.2
pandocfilters==1.4.2
parso==0.6.2
pexpect==4.8.0
pickleshare==0.7.5
prometheus-client==0.7.1
prompt-toolkit==3.0.4
ptyprocess==0.6.0
Pygments==2.6.1
pymongo==3.10.1
pyrsistent==0.15.7
python-dateutil==2.8.1
pyzmq==19.0.0
qtconsole==4.7.1
QtPy==1.9.0
s3transfer==0.3.3
scikit-learn==0.22.2.post1
scipy==1.4.1
Send2Trash==1.5.0
simplejson==3.17.0
six==1.14.0
sklearn==0.0
terminado==0.8.3
testpath==0.4.4
tornado==6.0.4
traitlets==4.3.3
urllib3==1.25.8
w

In [14]:
%%bash
set -e
source data/env-folder/bin/activate
pip wheel -w data/wheelhouse -r data/requirements.txt

Looking in links: /Users/rick446/src/wheelhouse, /Users/rick446/src/wheelhouse
Processing /Users/rick446/src/wheelhouse/appnope-0.1.0-py2.py3-none-any.whl
  Saved /Users/rick446/src/arborian-classes/data/wheelhouse/appnope-0.1.0-py2.py3-none-any.whl
Processing /Users/rick446/src/wheelhouse/attrs-19.3.0-py2.py3-none-any.whl
  Saved /Users/rick446/src/arborian-classes/data/wheelhouse/attrs-19.3.0-py2.py3-none-any.whl
Processing /Users/rick446/src/wheelhouse/backcall-0.1.0-py3-none-any.whl
  Saved /Users/rick446/src/arborian-classes/data/wheelhouse/backcall-0.1.0-py3-none-any.whl
  Using cached bleach-3.1.3-py2.py3-none-any.whl (151 kB)
  Saved /Users/rick446/src/arborian-classes/data/wheelhouse/bleach-3.1.3-py2.py3-none-any.whl
  Using cached boto3-1.12.26-py2.py3-none-any.whl (128 kB)
  Saved /Users/rick446/src/arborian-classes/data/wheelhouse/boto3-1.12.26-py2.py3-none-any.whl
  Using cached botocore-1.15.26-py2.py3-none-any.whl (6.0 MB)
  Saved /Users/rick446/src/arborian-classes/data

In [15]:
ls data/wheelhouse

Jinja2-2.11.1-py2.py3-none-any.whl
MarkupSafe-1.1.1-cp37-cp37m-macosx_10_6_intel.whl
Pygments-2.6.1-py3-none-any.whl
QtPy-1.9.0-py2.py3-none-any.whl
Send2Trash-1.5.0-py3-none-any.whl
appnope-0.1.0-py2.py3-none-any.whl
attrs-19.3.0-py2.py3-none-any.whl
backcall-0.1.0-py3-none-any.whl
bleach-3.1.3-py2.py3-none-any.whl
boto3-1.12.26-py2.py3-none-any.whl
botocore-1.15.26-py2.py3-none-any.whl
decorator-4.4.2-py2.py3-none-any.whl
defusedxml-0.6.0-py2.py3-none-any.whl
docutils-0.15.2-py3-none-any.whl
entrypoints-0.3-py2.py3-none-any.whl
importlib_metadata-1.5.0-py2.py3-none-any.whl
ipykernel-5.2.0-py3-none-any.whl
ipython-7.13.0-py3-none-any.whl
ipython_genutils-0.2.0-py2.py3-none-any.whl
ipywidgets-7.5.1-py2.py3-none-any.whl
jedi-0.16.0-py2.py3-none-any.whl
jmespath-0.9.5-py2.py3-none-any.whl
joblib-0.14.1-py2.py3-none-any.whl
jsonschema-3.2.0-py2.py3-none-any.whl
jupyter-1.0.0-py2.py3-none-any.whl
jupyter_client-6.1.0-py3-none-any.whl
jupyter_console-6.1.0-py2.py3-none-any.whl
jupyter_core-

Now we can distribute the `data/wheelhouse` directory with our project and install everything from the wheelhouse and not fetch from PyPI:

In [16]:
%%bash
set -e
source data/env-folder-2/bin/activate
pip install --no-index -f data/wheelhouse -r data/requirements.txt

Looking in links: /Users/rick446/src/wheelhouse, data/wheelhouse, /Users/rick446/src/wheelhouse


# Cleaning up virtualenvs

Since a virtualenv is just a directory, we can 'clean it up' by removing the directory:

In [17]:
!rm -r data/env-folder data/env-folder-2 data/wheelhouse data/requirements.txt

# Lab

Open [virtualenv lab][virtualenv-lab]

[virtualenv-lab]: ./virtualenv-lab.ipynb